In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random

In [2]:
articles = pd.read_csv('articlesDataset/cleanDataArticles', delimiter=',', header=None)
tables = pd.read_csv('articlesDataset/cleanDataTables', delimiter=',', header=None)

In [3]:
formattedTables = tables.iloc[:,:].values
formattedArticles = articles.iloc[:,:].values

In [4]:
corpusTablesCaption = []
corpusTablesHeader = []
corpusTablesText = []

for tables in formattedTables:
    
    corpusTablesCaption.append(str(tables[1]))
    corpusTablesHeader.append(str(tables[2]))
    corpusTablesText.append(str(tables[3]))

In [83]:
corpusArticlesID = []
corpusArticlesText = []

for i in range(20):
    
    randomArticle = random.choice(formattedArticles)
    
    corpusArticlesID.append(randomArticle[0])
    corpusArticlesText.append(str(randomArticle[1]))

In [8]:
vectorizerCaption = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerHeader = TfidfVectorizer(stop_words='english', analyzer='word')
vectorizerText = TfidfVectorizer(stop_words='english', analyzer='word')

tablesVectorCaption = vectorizerCaption.fit_transform(corpusTablesCaption)
tablesVectorHeader = vectorizerHeader.fit_transform(corpusTablesHeader)
tablesVectorText = vectorizerText.fit_transform(corpusTablesText)

In [84]:
articlesVectorCaption = vectorizerCaption.transform(corpusArticlesText)
articlesVectorHeader = vectorizerHeader.transform(corpusArticlesText)
articlesVectorText = vectorizerText.transform(corpusArticlesText)

In [13]:
def getTotalTablesByArticle(idArticle):

    countTopTables = 0

    for tables in formattedTables:
    
        if tables[0] == int(idArticle):
        
             countTopTables = countTopTables + 1
    
    #print(str(idArticle) + '-' + str(countTopTables))
    
    return countTopTables

In [14]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
    
        #index = distanceVector.index(topkDistance)
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [15]:
def getAccuracy(idRankedTables, idQueryGoal):

    totalRanquedTables = len(idRankedTables)

    match = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            match = match + 1

    accuracy = match / totalRanquedTables
    
    return accuracy

In [85]:
accuracy = []

totalSelectedArticles = int(articlesVectorCaption.shape[0])

for i in range(totalSelectedArticles):
    
    distanceVectorCaption = pairwise_distances(articlesVectorCaption[i], tablesVectorCaption, metric='cosine')
    distanceVectorHeader = pairwise_distances(articlesVectorHeader[i], tablesVectorHeader, metric='cosine')
    distanceVectorText = pairwise_distances(articlesVectorText[i], tablesVectorText, metric='cosine')
    
    distanceSum = np.sum([distanceVectorCaption, distanceVectorHeader, distanceVectorText], axis=0)
    distanceAvg = np.mean([distanceVectorCaption, distanceVectorHeader, distanceVectorText], axis=0)
    distanceMax = np.amax([distanceVectorCaption, distanceVectorHeader, distanceVectorText], axis=0)
    
    idQueryGoal = int(corpusArticlesID[i])
    
    countTopTables = getTotalTablesByArticle(idQueryGoal)
    
    topK = nsmallest(countTopTables, distanceSum[0])
    
    idRankedTables = getIdRankedTalbes(topK,distanceSum[0])
    
    #print(idRankedTables)
    
    accuracy.append(getAccuracy(idRankedTables,idQueryGoal))

In [86]:
np.mean(accuracy)

0.03392857142857143

In [87]:
corpusArticlesID

[342780,
 234598,
 3630165,
 18664181,
 5001707,
 33198042,
 33142448,
 24654684,
 236613,
 986935,
 2194459,
 32511265,
 34364582,
 18752637,
 1725270,
 21063711,
 3489830,
 245765,
 1680161,
 876603]